## PROJET DE DEEP LEARNING : AUDIO BIRD CLASSIFICATION

## NOTEBOOK DU PROJET ET RESULTATS OBTENUS

In [79]:
# importer les librairies importantes 
import math, random

import pandas as pd
import numpy as np
from pathlib import Path
import os

import torch
import torchaudio
from torchaudio import transforms
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn.functional as F
from torch.nn import init
from torch import nn
import time
from tqdm.auto import tqdm

from IPython.display import Audio

> En deep learning, pour accélerer les calculs, nous devons posséder un GPU ( Graphical Processing Unit ). Un GPU est une composante électronique de notre pc qui permet d'accélerer le processus d'entrainement des données. Si le GPU n'est pas disponible, par défaut notre libraire de DL utilise le CPU ( Central Processing Unit ).



> Vérifions si Pytorch utilise bien un accélérateur qui sera le GPU de notre pc par défaut dénommé : mps. Si le GPU est bien disponible pour accélerer les calculs, notre commande va nous retourner True sinon ce sera False dans le cas contraire et ce sera le CPU qui sera utilisé.

In [80]:
torch.backends.mps.is_available()

True

    Un accélerateur de calculs est donc présent ce qui signifie que Pytorch utilise bien le GPU de notre PC 

> Pour pouvoir utiliser le GPU, nous devons configurer Pytorch de sorte a ce qu'il fasse tourner les tenseurs sur le GPU. Pour ce faire nous devons définir une variable nommée **device** qui va contenir le nom de notre GPU. De cette facon, nous allons faire tourner tout sur GPU pour accélerer les calculs au travers de notre variable **device**. 

In [81]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [82]:
device

'mps'

    Pytorch est maintenant configuré pour tourner les calculs sur un certain accélérateur qui s'appelle device qui se trouve etre mps qui est notre GPU sur pc. Pour faire tourner les calculs sur GPU, nous devons explicitement déclaré cela a Pytorch dans le code auquel cas nous serons toujours sur CPU

In [83]:
# Par exemple, définissons simplement un tenseur : 
a = torch.tensor([1,3,2])

In [84]:
a    # notre tenseur a ne tourne pas sur GPU mais sur CPU

tensor([1, 3, 2])

In [85]:
# Essayons maintenant ce code 
b = torch.tensor([1,3,2]).to(device)

In [86]:
b   # notre tenseur tourne bien sur GPU et non pas sur CPU 

tensor([1, 3, 2], device='mps:0')

In [87]:
# Essayons de faire une opération pour voir ce que cela donne : 

a + b # Comme nous le voyons, les deux tenseurs opèrent sur deux différents 'devices', l'un sur CPU et l'autre GPU

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, mps:0 and cpu!

    Dans toute la suite de notre projet, nous ferons tourner tous nos calculs sur GPU pour accélerer les calculs

## PREPROCESSING DES DONNÉES 

    Nous allons définir maintenant via les librairie os et pathlib le répertoire de travail pour pouvoir importer nos données

In [88]:
os.chdir('/Users/nacersere/') # nous définissons notre répertoire de travail pour faciliter l'importation des données

In [89]:
download_path = Path.cwd()/'Downloads'

In [90]:
metadata_file = download_path/'ff1010bird_metadata.csv'

> Nous importons tout d'abord la métadonnée qui contient tous les labels, donc les variables a prédire des audios du fichier central téléchargé sur le site. C'est donc une information éssentielle pour notre projet.

In [91]:
df = pd.read_csv(metadata_file)

In [92]:
df.head()   # 5 premières lignes 

,itemid,hasbird
0,64486,0
1,2525,0
2,44981,0
3,101323,0
4,165746,0


In [93]:
df.tail()  # 5 dernières lignes 

,itemid,hasbird
7685,168059,0
7686,164922,0
7687,80789,1
7688,104733,1
7689,40565,0


> A partir de cette étape, nous allons adopter une stratégie qui nous permet d'importer nos données de manière efficiente. Nous savons que la base de donnée qui contient nos audios téléchargés est assez volumineuse : 6.5 GB. Importer toute la base de donnée directement sur Pandas est inefficient : Long temps de chargement et de calculs & consommation de mémoire inutile. 

> Par contre, il existe une manière bien plus efficiente d'importer la base donnée : une approche par **lien de répertoire**. Elle consiste a définir dans notre métadonnée importée, une nouvelle colonne contenant pour chaque observation de la métadonnée et donc de chaque label (0 ou 1) par la meme occasion, un lien de répertoire qui servirait a **référencer** l'audio correspondant dans le gros fichier téléchargé. Ainsi, plutot que d'importer toute la base audio, lors du processing, Python va chercher chaque audio dans le gros fichier au travers du **lien de répertoire** soigneusement défini au préalable, et le processer. On importe donc de manière **séquentielle** et non pas de manière linéaire nos audios. 


> Remarquons cependant que l'identifiant des audios dans la métadonnée (colonne itemid) correspond a l'identifiant des audios du gros fichier téléchargé sans l'extension .wav .Pour aider Python a réferencer chaque audio via un lien, nous procédons comme suit : 

   - Convertissons la colonne itemid en chaine de caractères
   - Nous ajoutons a cette colonne un '/' pour créer un chemin de répertoire 
   - Nous lui ajoutons enfin une extension .wav a la fin ( crucial ! ) car cela permet de référencer directement les audios depuis la base de donnée volumineuse. De cette manière on se retrouve avec une métadonnée qui contient pour chaque audio de la base de donnée au travers de son lien correspondant, le label qui lui sied.

In [94]:
df.dtypes    # Commande qui retourne le type des variables de notre dataframe

itemid     int64
hasbird    int64
dtype: object

In [95]:
df['itemid'] = df['itemid'].astype('str') # convertion de la colonne en chaine de caractère 

In [96]:
df.dtypes

itemid     object
hasbird     int64
dtype: object

In [97]:
df['relative_path'] = '/' + df['itemid'] + '.wav' 

    relative_path contient maintenant, pour chaque label (0 ou 1), le lien du fichier audio correspondant 

In [98]:
df.head()

,itemid,hasbird,relative_path
0,64486,0,/64486.wav
1,2525,0,/2525.wav
2,44981,0,/44981.wav
3,101323,0,/101323.wav
4,165746,0,/165746.wav


In [99]:
df = df[['relative_path', 'hasbird']]  # nous gardons seulement le label & relative_path dans le dataframe

In [100]:
df.head()

,relative_path,hasbird
0,/64486.wav,0
1,/2525.wav,0
2,/44981.wav,0
3,/101323.wav,0
4,/165746.wav,0


In [126]:
df.tail()

,relative_path,hasbird
7685,/168059.wav,0
7686,/164922.wav,0
7687,/80789.wav,1
7688,/104733.wav,1
7689,/40565.wav,0


> Une fois trouvée une manière d'importer les données, passons maintenant au préprocessing. Nous devons préprocesser nos audios avant de les introduire dans un réseau de neurones. Voici comment nous avons procéder étape par étape : 

- Nous avons créer une classe AudioUtil qui contient toutes nos étapes de Préprocessing des audios. Nous avons jugé bon d'utliser de l'orienté objet car cela est en accord avec Pytorch et cela nous permet d'assurer une certaine scalabilité et une certaine reproductibilité du code. Ainsi nous allons juste instancier, passer les paramètres, ce qui réduit consdérablement le temps de développement.

                Cette classe contient les fonctions de preprocessing suivantes par ordre : 
 
- La fonction **open()** : La première étape de preprocessing est la **numérisation des fichiers audios** ou le fait de transformer les audios du fichier volumineux en nombres puis au travers d'un modèle de DL, trouver des relations dans ces nombres. Pour cela nous utilisons une fonction **open()**, qui prend en paramètre le lien du répertoire de la métadonée et convertit au travers d'une fonction de Pytorch, **torchaudio.load()**, l'audio correspondant dans le fichier volumineux en **torch.tensor()**, l'équivalent des **numpy.array()** en quelque sortes. Autrement dit, **torchaudio.load()** convertit les audios en tableaux de nombres. la fonction **torch.load()** retourne deux variables importantes toutes contenues dans un tuple : 

> Le premier élément du tuple est l'équivalent numérique des fichiers audios. Autrement dit, ce sont les audios mais représenté sous forme de tableaux de nombres. 
   
> Le second élément tout aussi important est le **sample rate**. Vu que les audios par définition sont des ondes qui se déplacent, et qu'une onde a une **amplitude** et une **fréquence** donnée, le **sample rate** est donc le nombre d'amplitudes que l'on a du calculer par seconde pour convertir notre audio en tableaux de nombres. Autrement dit, un audio de 00:10 secondes aura une dimension de 441.000 puisque par seconde d'enregistrement, ce sont 44.100 amplitudes qui sont calculées (44.100) est la valeur par défaut couramment utilisé en théorie du signal et machine learning. C'est la valeur par défaut utilisée par la libraire torchaudio de Pytorch.


- La fonction **rechannel()** : Cette fonction permet de convertir des audios monos en audios stéréos et inversement. En effet, certains audios sont enregistrés en mono et d'autres enregistrés en stéréo. Vu que notre modèle de DL s'attend a ce que tous les tenseurs aient la meme dimension, il est important de convertir tous les audios sur la meme chaine : mono ou stéréo. Dans la suite de notre projet, nous avons décidé de convertir tous les audios en **stéréo**. 


- La fonction **resample()** : Cette fonction permet d'uniformiser le sample rate des audios pour que leurs représentations numériques aient la meme dimension. En effet, certains audios sont enregistrés avec un sample rate de par exemple 50.000. Cela signifie que le tableau de nombre équivalent a un audio de 00:10 secondes enregistré avec un sample rate de 50.000 sera de dimension 500.000 alors que celui d'un meme audio de 00:10 secondes enregistré avec un sample rate standard de 44.100 sera de dimension 441.000. Comment alors uniformiser les dimensions pour notre résau de neurones et éviter une erreur de dimenion ?. C'est la ou vient notre fonction **resample()**.  


- La fonction **pad_trunc()** : Cette fonction nous permet d'uniformiser cette fois **la durée des audios**. En effet, un tableau numérique d'un audio de 00:10 secondes enregistré avec un **sample rate** de 44.100 secondes aura une dimension de 441.000 alors qu'un audio de 00:20 secondes enregistrés avec un **sample rate** de 44.100 secondes aura une dimension de 20 * 44.100, soit deux fois plus que le premier. La fonction **pad_trunc** nous permet donc soit d'augmenter la durée des audios en ajoutant du silence soit en le diminuant. 

> En réalité, vu que nos audios sont tous de 00:10 secondes dans le fichier volumineux, cette étape de préprocessing n'était pas nécessaire puisque par défaut tous les audios sur lesquels nous travaillons aurons la meme dimension. Seul le **sample rate** peut varier, mais vu que en machine learning nous devons garder la généralisation d'un modèle en tete lors de sa conception et que nous ne savons pas comment et dans quelles condition seront les données de test, il est important de tout incorporer soigneusement. 
    
    
- La fonction **time_shift()** : Une technique courante pour **augmenter la diversité** d'un ensemble de données, en particulier lorsque l'on ne dispose pas de suffisamment de données, consiste à **augmenter artificiellement** nos données. Pour ce faire, nous **modifions légèrement** les échantillons de données existants. Par exemple, avec les images, nous pouvons faire des choses comme faire légèrement pivoter l'image, la recadrer ou la redimensionner, modifier les couleurs ou l'éclairage, ou ajouter du bruit à l'image. Étant donné que la sémantique de l'image n'a pas changé de manière significative, la même étiquette cible de l'échantillon d'origine s'appliquera toujours à l'échantillon augmenté. Par exemple, si l'image était étiquetée comme un "chat", l'image augmentée sera également un "chat". **Mais, du point de vue du modèle, cela ressemble à un nouvel échantillon de données.** Cela aide notre modèle à généraliser à une plus grande gamme d'entrées d'image. Tout comme pour les images, **il existe plusieurs techniques pour augmenter également les données audio**. Cette augmentation peut être faite à la fois sur l'audio brut avant de produire le spectrogramme, ou sur le spectrogramme généré. L'augmentation du spectrogramme produit généralement de meilleurs résultats.


> Le **time_shift()** est par conséquent une astuce classique de Preprocessing en classification audio consistant a augmenter les données sur le signal audio brut en lui appliquant un décalage temporel pour décaler l'audio vers la gauche ou la droite d'une quantité aléatoire. Cette augmentation crée artificiellement une diversité dans nos données et permet a notre modèle de généraliser plus facilement. Cet décalage permet d'avoir de très bons résultats comme on le verra par la suite et est une astuce parmi plusieurs autres. La fonction **time_shift** permet par conséquent d'augmenter artificellement nos données


- La fonction **specto_gram()** : Cette fonction nous permet de convertir ensuite l'audio augmenté en un spectrogramme. Ils capturent les caractéristiques essentielles de l'audio et constituent souvent le moyen le plus approprié d'entrer des données audio dans des modèles de DL. Le spectogram est par définition basique,la représentation la plus fidèle sous forme d'image de ce que serait les audios dans le vide. C'est un graphe qui pour chaque durée de l'audio, on lui fait correspondre la fréquence correspondante. Le spectogram étant sous forme d'image, **il convient parfaitement a un réseau de neurones convolutionnels**. La fonction **spectro_gram** permet de convertir nos audios augmentés en image. 


- La fonction **spectro_augment()** : Maintenant, nous pouvons faire une autre série d'augmentations, cette fois sur le Spectrogram plutôt que sur l'audio brut. L'objectif étant le meme que l'augmentation du fichier audio. Astuce classique en machine learning pour la performance et le pouvoir de généralisation.

In [125]:
class AudioUtil():
    
    @staticmethod
    def open(audio_file):
        sig, sr = torchaudio.load(audio_file)
        return (sig, sr)
    
    @staticmethod
    def rechannel(aud,new_channel):
        sig, sr = aud
        
        if (sig.shape[0] == new_channel):
            return aud 
        
        if (new_channel == 1):
            resig = sig[:1,:]
            
        else:
            resig = torch.cat([sig,sig])
            
        return ((resig, sr))
    
    @staticmethod
    def resample(aud, newsr):
        sig, sr = aud
        
        if (sr == newsr):
            return aud
        
        num_channels = sig.shape[0]
        
        resig = torchaudio.transforms.Resample(sr,newsr)(sig[:1,:])
        
        if (num_channels > 1):
            
            retwo = torchaudio.transforms.Resample(sr,newsr)(sig[1:,:])
            resig = torch.cat([resig,retwo])
        
        return ((resig,newsr))
    
    
    @staticmethod
    def pad_trunc(aud,max_ms):
        sig, sr = aud
        num_rows, sig_len = sig.shape
        max_len = sr//1000 * max_ms
        
        if (sig_len > max_len):
            sig = sig[:,:max_len]
            
        elif (sig_len < max_len):
            pad_begin_len = random.randint(0, max_len - sig_len)
            pad_end_len = max_len - sig_len - pad_begin_len 
            
            pad_begin = torch.zeros((num_rows, pad_begin_len))
            pad_end = torch.zeros((num_rows, pad_end_len))
            
            sig = torch.cat((pad_begin, sig, pad_end), 1)
            
        return (sig, sr)
    
    
    @staticmethod
    def time_shift(aud,shift_limit):
        sig, sr = aud
        _,sig_len = sig.shape
        shift_amt = int(random.random() * shift_limit * sig_len)
        
        return (sig.roll(shift_amt), sr)
    
    
    @staticmethod
    def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
        sig, sr = aud
        top_db = 80
        
        spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)
        
        spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
        
        return (spec)
    
    
    @staticmethod
    def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
        _, n_mels , n_steps = spec.shape
        mask_value = spec.mean()
        aug_spec = spec
        
        freq_mask_param = max_mask_pct * n_mels
        for _ in range(n_freq_masks):
            aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)
            
            time_mask_param = max_mask_pct * n_steps
            
        for _ in range(n_time_masks):
            aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)
            
            return aug_spec

> Une fois le preprocessing effectué, nous allons créer une classe **SoundDs** en héritant de la classe **Dataset** de Pytorch pour permettre d'appeler chaque fonction de préprocessing décrite précédemment tout en faisant passer les paramètres qui **caractérisent nos données**. 


> Notre idée est la suivante. Nous allons créer la classe **SoundDS** pour importer les audios proprement dit au travers des liens de répertoire et les processer au fur et a mésure en appelant les fonctions de notre classe **AudioUtil**. Nous allons par conséquent processer de manière **séquentielle** chaque audio, au travers de notre nouvelle classe. Voici comment nous procédons : 

- La programmation orientée objet utilisée précedemment dans le preprocessing prend tout son sens. En effet, il nous suffit d'appeler maintenant juste la classe précédente tout en passant dans les méthodes de cette classe les paramètres qui nous conviennent et qui sont propres a nos données. Cela nous génère un gain de temps en développement énorme, et permet ainsi de maintenir le code sur la durée et de ne pas revenir fréquemment regarder les paramètres des fonctions pour faire passer les arguments. On définit par conséquent la classe **SoundDS**


- On crée les fonctions habituelles comme dans une programmation orientée objet classique avec la fonction $__init__()$ . On utilise la fonction $__len__()$ qui est une surcharge d'opérateur pour nous permettre d'utiliser la fonction **len()** sur l'objet que l'on va créer au travers de notre classe **SoundDS** sous Python. Cet surcharge d'opérateur sera importante car elle nous permettra de spliter notre base de donnée en base de donnée d'entrainement et de test et meme de faire plus tard de la validation corisée. On utilise également la fonction $__getitem__()$ qui nous permet, de la meme manière que l'on utilise sur les listes, les numpy arrays ou les tenseurs, d'utiliser les $[]$ sur cette fois l'objet que l'on créer au travers de notre classe **SoundDS**


- Enfin, on instancie notre classe **AudioUtil** avec les paramètres caractéristiques de nos données:

    * **AudioUtil.open('/..../')** prend en paramètre le lien de répertoire des audios
   
    * **AudioUtil.resample(aud, self.sr)** prend en paramètres : le **résultat** de la fonction AudioUtil.open() qui est représenté par le paramètre **aud**. **aud** est en fait un tuple qui contient rappelons le deux résultats : La réprésentation sous forme de tableau des audios et le **sample rate avec lequel les audios ont été nativement enregistrés et dont Pytorch s'est servi pour encoder automatiquement ces derniers**. Vu que ces **sample rate natifs** peuvent différer, la fontion AudioUtil.resample() uniformise donc la représentation des audios sous forme de nombres issus de la fonction AudioUtil.open() avec le meme **sample rate** et ce **sample rate** est le paramètre **self.sr**. Ce **sample rate** sera le **sample rate** par défaut en pratique : 44.100.  A cette étape donc, tous nos fichiers audios sont encodés numériquement sous forme de tableau et ont la meme dimension.
   
    * **AudioUtil.rechannel(reaud, self.channel)** processe les audios issus du résultat de la fonction **AudioUtil.resample()** (qui est contenu dans la variable **reaud**), donc des fichiers audios ayant la meme dimension et encodés numériquement en les mettant tous sous forme **stéréo**. Tous nos audios qu'ils soient en **mono** ou **stéréo** seront tous transformés en **stéréo** au travers de la variable **self.channel**. Lorsque **self.channel** est égale a 2, nous sommes en stéréo. Lorsque c'est 1 nous sommes en mono.
    
    * **AudioUtil.pad_trunc(rechan, self.duration)** processe les résulats issus de **AudioUtil.rechannel()** au travers de la variable **rechan** en convertissant tous les audios pour qu'ils aient la meme durée. Vu que nos audios avaient déjà tous la meme durée, 00:10 secondes, cette étape n'est donc pas nécéssaire mais prévient des données de test dont les longueurs audios pourraient différer, ce qui causerait des problèmes de dimensions aux tenseurs. **self.duration** est la durée pour laquelle nous voulons que tous nos audios soient de meme durée, dans notre cas 10.000 milliseconds donc 10 seconds car les audios de la base volumineuse le sont tous. Cela nous donne par conséquent le meme résultat que **AudioUtil.rechannel()** puisqu'aucune transformation tangible a été nécessaire.
    
    * **AudioUtil.time_shift(dur_aud, self.shift_pct)** : Meme principe : on prend les résultats de la fonction précédente et on les processe. Ainsi on s'assure que l'on fait de manière séquentielle et que l'on oublie aucune étape de préprocessing. **dur_aud** est le résultat de la fonction **AudioUtil.pad_trunc()**. Par contre **self.shift_pct()** est le pourcentage pour lequel nous voulons shifter nos audios soit vers la gauche ou la droite pour créer de la diversité dans nos données et donc accroitre le pouvoir de généralisation de notre modèle. Elle sera égale a 0.4 soit 40% . C'est une valeur que nous avons pris juste de manière arbitraire. Vu qu'il doit etre décidé avant de faire tourner le modèle, c'est donc un **hyperparamètre**.
    
    * **AudioUtil.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)** : Prend les résultats de **AudioUtil.time_shift()** et les processe. L'idée est de transformer nos audios en spectograms donc en images, pour les faire passer au travers d'un CNN et trouver des relations intéressantes. **n_mels** est le nombre de bandes fréquentes en mels, donc la taille du spectogram. En réalité, les spectograms bruts issus des données audios ne sont pas chaque fois intéréssantes. En effet, souvent l'image générée par ce spectogram est très invisible, ce qui peut etre problématique pour notre modèle de DL.Comment reconnaitre comment se comporte un audio si son spectogram est illisible ? L'idée donc que les chercheurs ont eu est d'introduire ce que l'on appelle **spectogram de mel** d'ou le nom du paramètres **n_mels**. Le spectogram de mel est un spectogram qui, plutot que d'utiliser les fréquences des audios sur l'axe y et le temps sur l'axe x, utilise **l'échelle de mel** sur l'axe y et le temps sur l'axe x, ce qui donne des résultats très intéréssants et raisonnables.**n_fft** est la fenetre de temps. **n_mels** et **n_fft** étant définis a l'avance, ce sont des **hyperparamètres**
    
    Un article intéressant dont nous nous sommes servis pour comprendre l'échelle de mel est au lien suivant : https://fr.wikipedia.org/wiki/Échelle_des_mels.
    
    * **AudioUtil.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)** : Prend les résultats de la fonction **AudioUtil.spectro_gram()** qui est contenu dans **sgram** et le procèsse. A cette étape, nos audios sont sous forme de spectogram de mels. Mais vous voulons créer des **spectograms augmentés**.Tout comme le **time_shift()**, nous voulons créer de la diversité dans nos données et qui se traduiront par un modèle plus robuste et un bon pouvoir généralisateur. Cette augmentation se fait au travers des variables **n_freq_masks** ou masque de fréquence et de **n_time_masks** ou masque de temps. **n_freq_masks** masque au hasard une plage de fréquences consécutives en ajoutant des barres horizontales sur le spectrogramme tandis que **n_time_masks** similaire aux masques de fréquence,bloquent au hasard des plages de temps du spectrogramme en utilisant des barres verticales. **n_freq_masks** et **n_time_masks** étant définis avant de tourner le modèle, ce sont également des **hyperparamètres**

In [291]:
class SoundDS(Dataset):
  def __init__(self, df, data_path):
    self.df = df
    self.data_path = str(data_path)
    self.duration = 10000
    self.sr = 44100
    self.channel = 2
    self.shift_pct = 0.4
            
  # ----------------------------
  # Nombre d'éléments dans la base de donnée 
  # ----------------------------
  def __len__(self):
    return len(self.df)    
    
  # ----------------------------
  # Avoir le ième élément 
  # ----------------------------
  def __getitem__(self, idx):
    # Chemin de fichier absolu du fichier audio - concaténez le répertoire audio avec
    # le relative path
    audio_file = self.data_path + self.df.loc[idx, 'relative_path']
    # Obtenir l'ID de la Classe
    class_id = self.df.loc[idx, 'hasbird']

    aud = AudioUtil.open(audio_file)
    # Certains sons ont un taux d'échantillonnage plus élevé ou moins de canaux par rapport a la
    # majorité. Donc, faisons en sorte que tous les sons aient le même nombre de canaux et le même
    # taux d'échantillonnage. À moins que la fréquence d'échantillonnage ne soit la même, le pad_trunc continuera a
    # donner des tableaux de longueurs différentes, même si la durée du son est
    # le même.
    reaud = AudioUtil.resample(aud, self.sr)
    rechan = AudioUtil.rechannel(reaud, self.channel)

    dur_aud = AudioUtil.pad_trunc(rechan, self.duration)
    shift_aud = AudioUtil.time_shift(dur_aud, self.shift_pct)
    sgram = AudioUtil.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
    aug_sgram = AudioUtil.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)

    return aug_sgram , class_id

> Nous passons maintenant dans notre classe SoundDS, le dataframe représentant la métadonnée et lien de répertoire vers le fichier volumineux contenant les fichiers audio. 

In [104]:
my_sounds = SoundDS(df,'/Users/nacersere/Downloads/wav') # wav représente le fichier volumineux des audios

> Comme nous pouvons le voir, my_sounds est un objet crée via une classe. Lorsque l'on essaie d'afficher cet objet, Pyton nous dit qu'il s'agit d'un objet de type SoundDS stocké quelque part dans la mémoire. 

In [130]:
my_sounds   

> Lorsque nous utilisons la fonction len() sur notre objet de type SoundDS, il nous retourne un résultat or nous savons que la fonction len() fonctionne pour un certain type de données en Python, mais pourquoi fonctionne t'il sur un objet de type SoundDS, objet crée de manière artificielle et qui n'existait pas auparavant?. Cela est du a la surcharge des opérateurs. C'est la fonction $__len__()$ dans la classe SoundDS qui permet d'avoir ce résultat. On voit bien que l'on a 7690 données audios disponibles.

In [129]:
len(my_sounds)

7690

> Nous allons a présent essayer de voir ce qui se passe a l'intérieur de notre objet my_sounds. Tout comme avec la fonction len(), les $[]$ sont uilisés avec un certain type de données spécifiques en Python. Mais pourquoi un objet de type SoundDS fonctionne t'il avec les crochets? Cette propriété est du aussi a la surchage des opérateurs. Le fait d'étendre les opérateurs de base fondamentaux réservés pour un certain type de donnée a un autre type de donnée. Ainsi, grace a la fonction $__getitem__()$ de la classe SoundDS, nous pouvons voir le 8ième élément de nos données

> En indexant notre objet my_sounds, nous avons comme résultat un tuple. Le premier élément du tuple est un tensor qui représente le **spectogram augmenté** obtenu lors de notre préprocessing sous forme numérique avec des chiffres et le second élément est le label de l'audio pour savoir si oui ou non il contient des cris d'oiseaux. 0 pour Non et 1 pour oui.  

In [128]:
my_sounds[9]

(tensor([[[ 33.4620,  35.1140,  30.1704,  ...,  34.4985,  35.7045,  20.2986],
          [ 34.2874,  33.0401,  35.6473,  ...,  29.0326,  33.0858,  26.6811],
          [ 27.6349,  26.3337,  33.3903,  ...,  24.0278,  28.5904,  25.5299],
          ...,
          [-19.7180, -23.8581, -23.3542,  ..., -23.3197, -20.3997, -22.7305],
          [-21.1493, -22.7330, -21.9534,  ..., -24.3419, -22.9597, -23.0070],
          [-23.6225, -23.4210, -24.4804,  ..., -24.4417, -24.8352, -25.1972]],
 
         [[ 33.4620,  35.1140,  30.1704,  ...,  34.4985,  35.7045,  20.2986],
          [ 34.2874,  33.0401,  35.6473,  ...,  29.0326,  33.0858,  26.6811],
          [ 27.6349,  26.3337,  33.3903,  ...,  24.0278,  28.5904,  25.5299],
          ...,
          [-19.7180, -23.8581, -23.3542,  ..., -23.3197, -20.3997, -22.7305],
          [-21.1493, -22.7330, -21.9534,  ..., -24.3419, -22.9597, -23.0070],
          [-23.6225, -23.4210, -24.4804,  ..., -24.4417, -24.8352, -25.1972]]]),
 0)

In [131]:
my_sounds[9][0]  # 1er élément du tuple

tensor([[[ 31.8502,  32.7419,  24.4008,  ...,  36.4841,  34.1769,  26.6737],
         [ 30.4974,  28.3375,  29.1493,  ...,  32.5962,  35.4485,  20.6440],
         [ 31.3923,  31.3897,  29.0918,  ...,  24.9208,  30.0547,  18.2222],
         ...,
         [-22.6661, -23.2208, -23.0941,  ..., -22.4938, -22.7700, -22.2436],
         [-25.6760, -23.9084, -22.4351,  ..., -21.3864, -22.6048, -23.2098],
         [-24.4264, -23.9438, -23.3493,  ..., -23.4859, -24.1167, -24.6893]],

        [[ 31.8502,  32.7419,  24.4008,  ...,  36.4841,  34.1769,  26.6737],
         [ 30.4974,  28.3375,  29.1493,  ...,  32.5962,  35.4485,  20.6440],
         [ 31.3923,  31.3897,  29.0918,  ...,  24.9208,  30.0547,  18.2222],
         ...,
         [-22.6661, -23.2208, -23.0941,  ..., -22.4938, -22.7700, -22.2436],
         [-25.6760, -23.9084, -22.4351,  ..., -21.3864, -22.6048, -23.2098],
         [-24.4264, -23.9438, -23.3493,  ..., -23.4859, -24.1167, -24.6893]]])

In [132]:
my_sounds[9][1]   # second element du tuple

0

> On voit clairement que pour chaque fichier audio, nous avons le label correspondant qui va avec et le preprocessing qui a été effectué de manière **séquentielle** et non **linéaire**. Nous sommes enfin pret a passer a la modélisation : Le modèle de DL en Pytorch

## MODÈLE CNN DE DEEP LEARNING AVEC PYTORCH

> Dans la suite de notre projet, nous avons adopté deux approches : 

- Nous allons spliter nos données en deux : 80% de données en entrainement et 20% en test, puis nous allons calculer sur chaque portion (entrainement et test), la **loss** et l'**accuracy** pour voir si notre modèle performe bien 



- Nous allons utiliser dans une seconde approche, une validation croisée pour confirmer les performances du modèle.


> Voici comment nous procédons : 

- Nous prenons le nombre d'observations totales de nos données audio
- Nous multiplions la taille de nos données par 0.8 en arrondissant, ce qui nous 80% des observations totales
- Nous calculons la portion restante entre le nombre total des données et celui des 80% on obtient celui des 20% des observations totales 
- Nous utilisons l'équivalent de la fonction **train_test_split()** en **sklearn** sous Pytorch qui est **random_split**, tout en prenant soin de spécifier la graine du générateur de nombres aléatoires pour assurer la reproductibilité en le fixant a 42. Nous passons a cette fonction l'objet my_sounds et la portion des données d'entrainement et de test sous forme de liste en prenant soin de conserver l'ordre. 
- Pytorch nous retourne donc deux données : le jeu d'entrainement et le jeu de test.

> Une fois les jeux de données d'entrainement et de test obtenus, nous les chargeons sous forme de **dataloader** en Pytorch. 

> Le DataLoader fait ce que vous pensez qu'il pourrait faire. Il aide à charger des données dans un modèle. Pour l'entrainement et pour l'inférence. Il transforme un grand ensemble de données ou dataset en un itérable Python de plus petits morceaux. Ces petits morceaux sont appelés batchs ou mini-batchs et peuvent être définis par le paramètre batch_size. Pourquoi faire ceci? Parce que c'est plus efficace en termes de calcul. Dans un monde idéal, nous pourrions effectuer le forward pass et le backward pass sur toutes nos données à la fois. Mais une fois que nous commencons à utiliser des ensembles de données très volumineux, à moins que nous ne disposions d'une puissance de calcul infinie, il est plus facile de les diviser en lots. Cela donne également à notre modèle plus de possibilités d'amélioration. Avec les mini-batchs (petites portions de données), la descente de gradient est effectuée plus souvent par époque (une fois par mini-lot plutôt qu'une fois par époque).

In [105]:
num_items = len(my_sounds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(my_sounds, [num_train, num_val],generator=torch.Generator().manual_seed(42))



train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False)

> Une fois cette étape accomplie, nous allons définir maintenant le modèle de DL proprement dit en Pytorch. Cette classe sera appelée AudioClassifier. 

> Notre modèle consiste en l'utilisation d'un modèle CNN de 03 Blocks de convolution en uilisant une initialisation de Kaiming pour les poids. Cela a pour effet de faire converger plus rapidement le modèle. 

**Construction du modèle de Deep Learning**

In [106]:
class AudioClassifier (nn.Module):
    # ----------------------------
    # Architecture du modèle
    # ----------------------------
    def __init__(self):
        super().__init__()
        conv_layers = []

        # 1er Block de Convolution avec Relu et Batch Norm. Utilisation d'une Initialisatioin de  Kaiming
        self.conv1 = nn.Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        # Second Block de Convolution
        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        # Second Block de Convolution 
        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv3.weight, a=0.1)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        # Second Block de Convolution
        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv4.weight, a=0.1)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        # Classificateur linéaire 
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=64, out_features=10)

       
        self.conv = nn.Sequential(*conv_layers)
 
    # ----------------------------
    # Forward pass 
    # ----------------------------
    def forward(self, x):
        # Tourner les blocks convolutionnels
        x = self.conv(x)

        # Pool adaptatif et aplatir pour l'entrée dans la couche linéaire
        x = self.ap(x)
        x = x.view(x.shape[0], -1)

        # Couche linéaire 
        x = self.lin(x)

        # Résultat Final
        return x

> Puis on fait les étapes classiques de création d'un modèle sous Pytorch : 

- On instancie le modèle
- On configure le GPU
- On exporte le modèle sous GPU

In [107]:
myModel = AudioClassifier()
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
myModel = myModel.to(device)
next(myModel.parameters()).device

device(type='mps', index=0)

> Récapitulatif du modèle de DL sous Pytorch

In [108]:
myModel.parameters 

<bound method Module.parameters of AudioClassifier(
  (conv1): Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (relu1): ReLU()
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu2): ReLU()
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu3): ReLU()
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (relu4): ReLU()
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (ap): AdaptiveAvgPool2d(output_size=1)
  (lin): Linear(in_features=64, out_features=10, bias=True)
  (conv): Sequential(
    (0): Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
    (1): ReLU()
    

> Vu que nous sommes dans un problème de classification, il est important lors du Forward pass et du Backward pass de calculer la **loss** et l'**accuracy**

- Pour ce faire, nous avons écris des fonctions customisées permettant de calculer l'accuracy et la loss de notre modèle lors de chaque passage, donc de chaque epoch.

> Fonction customisée permettant de calculer l'accuracy de notre modèle de DL sous PyTorch

In [109]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()    # nous avons utilisé PyTorch et torch.eq()
    acc = (correct / len(y_pred)) * 100 
    return acc

> Fonction customisée permettant de calculer la loss de notre modèle de DL sous PyTorch

In [110]:
loss_fn = nn.CrossEntropyLoss()          # nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=myModel.parameters(), # nous avons utilisé PyTorch et torch.optim.SGD()
                            lr=0.1)

   *Nous avons utilisé un gradient stochastique car c'est avec lui que nous avons atteint la meilleure performance du modèle. Avec un learning rate de 0.1, nous avons performé très bien en termes de Loss et Accuracy. Avec un Adam et un learning rate de 0.01, nous avons fait tout aussi bien. La différence entre les deux modèles en termes de performance est de 1%. SGD surperforme Adam de 1%.  Tous les modèles ont été sauvegardés et seront fournis avec le notebook final.*

> L'étape la plus importante après avoir défini le modèle, la loss et l'accuracy, est de définir une fonction d'entrainement et de test PyTorch. L'avantage de fonctionaliser l'étape d'entrainement et de test PyTorch est que plutot que d'écrire les boucles d'entrainement et de test encore et encore, nous pouvons écrire une fonction en 1 seule fois et l'appeler a volonté pour entrainer/tester notre modèle. Voici comment nous avons procédé.

- Nous définissons une fonction train_step qui prend en entrée :
    * un modèle de type torch.nn.Module : notre modèle de DL dans de cas 
    * une donnée/data_loader de type torch.utils.data.DataLoader : nos données d'netrainement train_dl
    * une fonction loss de type torch.nn.Module : notre fonction loss_fn customisée PyTorch
    * une fontion accuracy : notre fonction customisée accuracy_fn PyTorch
    * et une device de type torch.device : Notre GPU
    
- Pour la définition de la fonction test_step, la logique est la meme que celle de la fonction train_step

- En écrivant nos fonctions nous avons privilégié des annotations de type. Les annotations de type Python permettent a l'utilisateur de savoir a l'avance en utilisant une fonction, quels sont les types des paramètres qui sont valides pour la fonction. Par exemple dans la fonction test_step, le modèle doit etre de type torch.nn.Module pour que la fonction puisse marcher.

In [111]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn,
               device: torch.device = device):
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(data_loader):
        # Envoyons X et y sur GPU
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calcul de la loss
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimisation zero_grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimisation step
        optimizer.step()

    # Calcul de la loss et accuracy par epoch et affichons ce qui se passe 
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn,
              device: torch.device = device):
    test_loss, test_acc = 0, 0
    model.eval() # configurer le modèle en mode evaluation
    # Configurons le torch inference context manager
    with torch.inference_mode(): 
        for X, y in data_loader:
            # Envoyons X et y sur GPU
            X, y = X.to(device), y.to(device)
            
            # 1. Forward pass
            test_pred = model(X)
            
            # 2. Calcul de la loss et de  l'accuracy
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )
    
        # Adjustement des metrics and affichage
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

> Après avoir défini des fonctions d'entrainement, nous pouvons passer a l'apprentissage proprement dit. Nous procédons comme suit : 

- Nous définissons le nombre d'épochs qui sera de 150.
- Nous appelons les fonctions train_step & test_step en faisant passer les paramètres correspondants. 
- Nous laissons le modèle apprendre sur GPU

**Apprentisage du modèle avec PyTorch**

In [46]:
torch.manual_seed(42)

# Mesure du temps
from timeit import default_timer as timer
train_time_start_on_gpu = timer()

epochs = 150
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train_step(data_loader=train_dl, 
        model=myModel, 
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn
    )
    test_step(data_loader=val_dl,
        model=myModel,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn
    )

train_time_end_on_gpu = timer()


  0%|          | 0/150 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 0.61031 | Train accuracy: 75.71%
Test loss: 0.51395 | Test accuracy: 76.10%

Epoch: 1
---------
Train loss: 0.49988 | Train accuracy: 77.22%
Test loss: 0.65322 | Test accuracy: 60.70%

Epoch: 2
---------
Train loss: 0.47145 | Train accuracy: 79.22%
Test loss: 0.47678 | Test accuracy: 78.93%

Epoch: 3
---------
Train loss: 0.45551 | Train accuracy: 80.11%
Test loss: 0.45443 | Test accuracy: 79.96%

Epoch: 4
---------
Train loss: 0.44304 | Train accuracy: 80.99%
Test loss: 0.46260 | Test accuracy: 79.77%

Epoch: 5
---------
Train loss: 0.42798 | Train accuracy: 82.09%
Test loss: 0.42512 | Test accuracy: 81.57%

Epoch: 6
---------
Train loss: 0.41745 | Train accuracy: 82.53%
Test loss: 0.42919 | Test accuracy: 81.44%

Epoch: 7
---------
Train loss: 0.40412 | Train accuracy: 83.43%
Test loss: 0.49962 | Test accuracy: 80.22%

Epoch: 8
---------
Train loss: 0.39393 | Train accuracy: 83.57%
Test loss: 0.43037 | Test accuracy: 82.47%

Epoch: 9
---------
Train los

Test loss: 0.39306 | Test accuracy: 86.34%

Epoch: 76
---------
Train loss: 0.26824 | Train accuracy: 89.89%
Test loss: 0.34518 | Test accuracy: 86.66%

Epoch: 77
---------
Train loss: 0.26908 | Train accuracy: 90.71%
Test loss: 0.35599 | Test accuracy: 86.86%

Epoch: 78
---------
Train loss: 0.26698 | Train accuracy: 89.82%
Test loss: 0.38408 | Test accuracy: 86.60%

Epoch: 79
---------
Train loss: 0.26747 | Train accuracy: 90.21%
Test loss: 0.37226 | Test accuracy: 86.34%

Epoch: 80
---------
Train loss: 0.26650 | Train accuracy: 90.32%
Test loss: 0.33192 | Test accuracy: 87.24%

Epoch: 81
---------
Train loss: 0.26081 | Train accuracy: 90.36%
Test loss: 0.34773 | Test accuracy: 87.82%

Epoch: 82
---------
Train loss: 0.25939 | Train accuracy: 90.36%
Test loss: 0.35220 | Test accuracy: 88.34%

Epoch: 83
---------
Train loss: 0.26196 | Train accuracy: 90.65%
Test loss: 0.35449 | Test accuracy: 87.56%

Epoch: 84
---------
Train loss: 0.26247 | Train accuracy: 90.28%
Test loss: 0.36707 

> Nous voyons que notre modèle performe très bien sur nos données. L'accuracy sur le jeu de test a chaque itération semble etre bon. Vérifions la performance globale de notre modèle. Pour cela, nous avons écris une fonction pour permet d'évaluer notre modèle. 

In [112]:
torch.manual_seed(42)
def eval_model(model: torch.nn.Module, 
               data_loader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               accuracy_fn, 
               device: torch.device = device):
    """Evaluates a given model on a given dataset.

    Args:
        model (torch.nn.Module): A PyTorch model capable of making predictions on data_loader.
        data_loader (torch.utils.data.DataLoader): The target dataset to predict on.
        loss_fn (torch.nn.Module): The loss function of model.
        accuracy_fn: An accuracy function to compare the models predictions to the truth labels.
        device (str, optional): Target device to compute on. Defaults to device.

    Returns:
        (dict): Results of model making predictions on data_loader.
    """
    loss, acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in data_loader:
            # Envoyons X et y sur GPU
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))
        
        # Scale loss and acc
        loss /= len(data_loader)
        acc /= len(data_loader)
    return {"model_name": model.__class__.__name__, # marche seulement si le modèle a été crée avec une classe
            "model_loss": loss.item(),
            "model_acc": acc}

> Notre modèle performe **très bien** en terme de performance. En effet, ce dernier a atteint un accuracy de 87.95% sur le jeu de test. Pour un modèle de deep learning et un simple CNN, c'est **très acceptable**. Nous rappelons que nous avons utilisé une initialisation de Kaiming des poids et nos fonctions de préprocessing semblent bien marcher sur nos audios : spectogram augmenté obtenu via spectogram de mel.

In [48]:
my_model_results = eval_model(model=myModel, data_loader=val_dl,
    loss_fn=loss_fn, accuracy_fn=accuracy_fn,
    device=device
)
my_model_results

{'model_name': 'AudioClassifier',
 'model_loss': 0.3632675111293793,
 'model_acc': 87.95103092783505}

> Cependant, lorsqu'un modèle de DL fonctionne bien et meme très bien, c'est souvent le signe d'un overfitting ou le fait de surapprendre les données donc le bruit a l'intérieur, ce qui réduit drastiquement son pouvoir de généralisation. Pour confirmer les résultas, nous allons faire une validation croisée avec PyTorch. 

**Cross Validation avec Pytorch**

> Nous allons utiliser une validation croisée de deep learning pour voir si notre modèle performe vraiment bien. Pour ce faire nous allons utiliser ce qui est courant en deep learning pour faire du deep learning : Nous allons subdiviser nos données en : 

- Un jeu d'entrainement : 80% de la base de donnée 
- un jeu de validation pour hyperparametrer le modèle : 10% de la base de donnée 
- et un jeu de test pour la performance finale : 10% de la base de donnée 

> L'idée est simple : Nous allons entrainer et hyperparamétrer notre modèle sur le jeu d'entrainement et de validation. Nous allons faire apprendre notre modèle le plus longtemps possible sur nos données pour avoir de bons paramètres. Et nous allons tester son pouvoir de généralisation sur le jeu de test pour voir si notre modèle n'overfit pas 

In [113]:
num_items_cross = len(my_sounds)
num_train_cross = round(num_items_cross * 0.8)   # entrainement
num_val_cross = round(num_items_cross * 0.1)     # validation
num_test_cross = round(num_items_cross * 0.1)    # test 

> Nous utilisons comme précédemment la fonction random_split pour spliter nos données en jeu d'entrainement, en jeu de validation et en jeu de test.

In [114]:
train_ds_cross, val_ds_cross, test_ds_cross = random_split(my_sounds, [num_train_cross, num_val_cross, num_test_cross],generator=torch.Generator().manual_seed(42))

> On les transforme en DataLoader avec **torch.utils.data.DataLoader** : étape classique en Pytorch lorsque l'on fait du deep learning

In [115]:
train_dl_cross = torch.utils.data.DataLoader(train_ds_cross, batch_size=16, shuffle=True)  # entrainement
val_dl_cross = torch.utils.data.DataLoader(val_ds_cross, batch_size=16, shuffle=False)   # validation
test_dl_cross = torch.utils.data.DataLoader(test_ds_cross,batch_size=num_test_cross)   # test

> On crée une fonction train_val_cross_step comme précédemment pour éviter d'écrire encore et encore du code pour faire apprendre le modèle de deep learning. 

In [116]:
def train_val_cross_step(model: torch.nn.Module,
               train_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               val_loader:torch.utils.data.DataLoader,
               device: torch.device = device):

  # nombre d'epochs
  epochs = 150


  # initialisation des losses 
  train_loss      = torch.zeros(epochs)
  val_loss        = torch.zeros(epochs)
  train_accuracy  = torch.zeros(epochs)
  val_accuraccy   = torch.zeros(epochs)


  # Boucler a travers les epochs 
  for epochi in tqdm(range(epochs)):
    
    print(f"Epoch: {epochi}\n---------")

    # Boucler a travers  les training data batches
    model.train() # configurer le modèle en mode entrainement 
    batch_loss = []
    batch_accuracy  = []
    for X,y in train_loader:

      # Envoyer X et y sur le GPU
      X = X.to(device)
      y = y.to(device)

      # forward pass et la loss
      yHat = model(X)
      loss = loss_fn(yHat,y)

      # backprop
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # loss et accuracy a travers chaque batch
      batch_loss.append(loss.item())
      batch_accuracy.append( torch.mean((torch.argmax(yHat,axis=1) == y).float()).item() )
    # fin de la boucle de chaque batch 

    # et obtention des loss moyennes et des accuracies sur toutes les batches
    train_loss[epochi] = np.mean(batch_loss)
    train_accuracy[epochi]  = 100*np.mean(batch_accuracy)
    
    

    #### Performance sur le jeu de test (fait ici dans les batches!)
    model.eval() # configuer le modèle en mode évaluation 
    batch_accuracy  = []
    batch_loss = []
    for X,y in val_loader:

      # Envoyer X et y sur GPU
      X = X.to(device)
      y = y.to(device)

      # forward pass et la loss
      with torch.no_grad():
        yHat = model(X)
        loss = loss_fn(yHat,y)
      
      # loss et accuracy a travers chaque batch
      batch_loss.append(loss.item())
      batch_accuracy.append( torch.mean((torch.argmax(yHat,axis=1) == y).float()).item() )
    # fin de la boucle de chaque batch 

    # et obtention des loss moyennes et des accuracies sur toutes les batches
    val_loss[epochi] = np.mean(batch_loss)
    val_accuraccy[epochi]  = 100*np.mean(batch_accuracy)

    print(f"Train loss: {train_loss[epochi]:.5f} | Train accuracy: {train_accuracy[epochi]:.2f}% | Validation loss: {val_loss[epochi]:.5f} | Validation accuracy: {val_accuraccy[epochi]:.2f}")

> Nous pouvons entrainer maintenant le modèle sur la base d'entrainement et de test.

In [117]:
train_val_cross_step(myModel,train_dl_cross,loss_fn,optimizer,val_dl_cross,'mps')   # apprentissage 

  0%|          | 0/150 [00:00<?, ?it/s]

Epoch: 0
---------
Train loss: 0.61701 | Train accuracy: 75.24% | Validation loss: 0.52636 | Validation accuracy: 75.89
Epoch: 1
---------
Train loss: 0.48487 | Train accuracy: 78.28% | Validation loss: 0.51428 | Validation accuracy: 77.42
Epoch: 2
---------
Train loss: 0.46798 | Train accuracy: 79.55% | Validation loss: 0.49224 | Validation accuracy: 76.91
Epoch: 3
---------
Train loss: 0.45574 | Train accuracy: 80.00% | Validation loss: 0.48620 | Validation accuracy: 77.30
Epoch: 4
---------
Train loss: 0.44166 | Train accuracy: 80.65% | Validation loss: 0.49647 | Validation accuracy: 78.83
Epoch: 5
---------
Train loss: 0.43643 | Train accuracy: 81.10% | Validation loss: 0.47622 | Validation accuracy: 79.34
Epoch: 6
---------
Train loss: 0.42732 | Train accuracy: 82.18% | Validation loss: 0.44783 | Validation accuracy: 80.61
Epoch: 7
---------
Train loss: 0.41603 | Train accuracy: 82.66% | Validation loss: 0.47604 | Validation accuracy: 79.72
Epoch: 8
---------
Train loss: 0.41704 |

Train loss: 0.29464 | Train accuracy: 88.83% | Validation loss: 0.35940 | Validation accuracy: 86.22
Epoch: 69
---------
Train loss: 0.28697 | Train accuracy: 89.11% | Validation loss: 0.36028 | Validation accuracy: 85.33
Epoch: 70
---------
Train loss: 0.28477 | Train accuracy: 89.40% | Validation loss: 0.35456 | Validation accuracy: 87.50
Epoch: 71
---------
Train loss: 0.29156 | Train accuracy: 89.33% | Validation loss: 0.35021 | Validation accuracy: 87.24
Epoch: 72
---------
Train loss: 0.28487 | Train accuracy: 89.42% | Validation loss: 0.41543 | Validation accuracy: 85.46
Epoch: 73
---------
Train loss: 0.28235 | Train accuracy: 89.25% | Validation loss: 0.40039 | Validation accuracy: 86.73
Epoch: 74
---------
Train loss: 0.29762 | Train accuracy: 88.86% | Validation loss: 0.38882 | Validation accuracy: 86.10
Epoch: 75
---------
Train loss: 0.28334 | Train accuracy: 89.69% | Validation loss: 0.36950 | Validation accuracy: 86.35
Epoch: 76
---------
Train loss: 0.28251 | Train accu

Train loss: 0.25538 | Train accuracy: 90.58% | Validation loss: 0.36716 | Validation accuracy: 86.61
Epoch: 137
---------
Train loss: 0.24966 | Train accuracy: 90.36% | Validation loss: 0.39819 | Validation accuracy: 86.22
Epoch: 138
---------
Train loss: 0.25518 | Train accuracy: 90.41% | Validation loss: 0.39678 | Validation accuracy: 86.73
Epoch: 139
---------
Train loss: 0.25494 | Train accuracy: 90.16% | Validation loss: 0.38438 | Validation accuracy: 85.59
Epoch: 140
---------
Train loss: 0.25102 | Train accuracy: 90.65% | Validation loss: 0.36299 | Validation accuracy: 87.37
Epoch: 141
---------
Train loss: 0.25458 | Train accuracy: 90.32% | Validation loss: 0.40253 | Validation accuracy: 85.97
Epoch: 142
---------
Train loss: 0.25950 | Train accuracy: 90.29% | Validation loss: 0.37908 | Validation accuracy: 86.73
Epoch: 143
---------
Train loss: 0.25158 | Train accuracy: 90.71% | Validation loss: 0.47466 | Validation accuracy: 84.82
Epoch: 144
---------
Train loss: 0.26024 | Tr

> Évaluons maintenant notre modèle sur la base de test pour voir la performance après que ce dernier aie appris sur le jeu d'entrainement et le jeu de validation

> Nous le faisons au travers d'une fonction customisée comme auparavant permettant de calculer la loss et l'accuracy sur la base de test

In [122]:
def test_cross_step(model: torch.nn.Module,
                   test_loader: torch.utils.data.DataLoader,
                   loss_fn: torch.nn.Module,
                   device: torch.device=device):
    
    model.eval()
    X,y = next(iter(test_loader))
    
    X = X.to(device)
    y = y.to(device)
    
    with torch.no_grad():
        yHat = model(X)
        loss = loss_fn(yHat,y)
    
    test_loss = loss.item()
    test_accuracy = 100 * torch.mean((torch.argmax(yHat,axis=1) == y).float()).item()
    
    print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_accuracy:.2f}%")

In [123]:
test_cross_step(myModel,test_dl_cross,loss_fn,device)

Test loss: 0.32197 | Test accuracy: 89.47%


> Notre modèle de deep learning performe **très très bien**. Comme nous pouvons le voir, la performance atteinte sur la base de test est de **89.47%** avec une cross-validation ! Par conséquent, notre modèle de DL n'overfit pas et apprend parfaitement bien les données. 

## PRÉDICTIONS DES AUDIOS DU JEU DE TEST A TELECHARGER SUR LE SITE WEB DU PROJET AVEC NOTRE MODÈLE DE DL

> Nous allons maintenant importer la métadonnée du jeu de test a télécharger sur le site et essayer de prédire les audios. 

In [269]:
df_test_predict = pd.read_csv('https://ia903201.us.archive.org/28/items/birdaudiodetectionchallenge_test/badch_testset_blankresults.csv')

In [270]:
df_test_predict.head()   # voyons les premières lignes de la base 

,itemid,hasbird
0,00016765-bad0-4e2c-ad4e,NaN
1,0005ae67-efdc-446f-aeee,NaN
2,000a3cad-ef99-4e5e-9845,NaN
3,00270340-40d5-4947-9255,NaN
4,002d0637-e8fb-44a8-916b,NaN


In [271]:
df_test_predict.tail()

,itemid,hasbird
8615,ffe73a17-84eb-4fe8-99e2,NaN
8616,fff2dd8e-6973-4a62-86fa,NaN
8617,fff8ec92-afc0-48dd-9bb9,NaN
8618,fff92255-4d84-4136-9c58,NaN
8619,fffc94c1-ff49-4860-b96e,NaN


> Comme nous le voyons la métadonnée du jeu de test sur le site web ne contient aucun label 'hasbird'. Pas de 0 ou de 1. Nous allons assayer de prédire ces labels du jeu de test avec notre modèle de DL. 

> Comme précedemment, nous allons essayer de référencer les audios de la base de donnée de test téléchargée sur le site via les liens de répertoire. C'est exactement la meme procédure que l'on a fait précédemment pour entrainer nos données.

In [272]:
df_test_predict.dtypes

itemid      object
hasbird    float64
dtype: object

In [273]:
df_test_predict['itemid'] = df_test_predict['itemid'].astype('str')

In [274]:
df_test_predict['relative_path'] = '/' + df_test_predict['itemid'] + '.wav'

In [275]:
df_test_predict.head()

,itemid,hasbird,relative_path
0,00016765-bad0-4e2c-ad4e,NaN,/00016765-bad0-4e2c-ad4e.wav
1,0005ae67-efdc-446f-aeee,NaN,/0005ae67-efdc-446f-aeee.wav
2,000a3cad-ef99-4e5e-9845,NaN,/000a3cad-ef99-4e5e-9845.wav
3,00270340-40d5-4947-9255,NaN,/00270340-40d5-4947-9255.wav
4,002d0637-e8fb-44a8-916b,NaN,/002d0637-e8fb-44a8-916b.wav


In [276]:
df_test_predict = df_test_predict[['relative_path']] # comme il n'ya pas de label, nous retenons seulement le 'relative_path'

In [277]:
my_sounds_predict = SoundDS(df_test_predict,'/Users/nacersere/Downloads/wav-2')

In [278]:
len(my_sounds_predict)

8620

In [283]:
my_sounds_predict[20]  # visualisation du spectogram augmenté du 19 ième audio 

tensor([[[12.3819,  2.1034,  5.4741,  ..., -0.0576, -6.2201, 10.9702],
         [12.8517, -2.8655,  2.6813,  ..., -0.7263, -0.8349, 11.1678],
         [14.0566, -0.4229,  5.5248,  ..., -2.2426,  0.9876,  9.5981],
         ...,
         [22.5322, 24.9996, 24.0414,  ..., 25.7002, 24.1045, 25.0527],
         [24.7540, 24.6198, 25.9706,  ..., 25.5673, 25.5841, 25.7302],
         [17.8433, 17.3420, 18.4870,  ..., 17.4362, 19.4744, 19.8112]],

        [[12.3819,  2.1034,  5.4741,  ..., -0.0576, -6.2201, 10.9702],
         [12.8517, -2.8655,  2.6813,  ..., -0.7263, -0.8349, 11.1678],
         [14.0566, -0.4229,  5.5248,  ..., -2.2426,  0.9876,  9.5981],
         ...,
         [22.5322, 24.9996, 24.0414,  ..., 25.7002, 24.1045, 25.0527],
         [24.7540, 24.6198, 25.9706,  ..., 25.5673, 25.5841, 25.7302],
         [17.8433, 17.3420, 18.4870,  ..., 17.4362, 19.4744, 19.8112]]])

In [284]:
my_sounds_predict[20].shape, my_sounds_predict[1000].shape  # Tous nos audios sont uniformes. 

(torch.Size([2, 64, 860]), torch.Size([2, 64, 860]))

In [285]:
preds = torch.utils.data.DataLoader(my_sounds_predict) # On charge les données audios dans le DataLoader PyTorch

> Après ces étapes de processing, nous allons créer une fonction pour prédire les différents audios au travers de notre modèle de DL. Voici comment nous allons procéder :

- Nous allons prédire les probabilités que dans un audio, il y ai un oiseau ou pas. Ces proba seront calculées avec la fonction **torch.softmax()**.  
- Puis, avec ces probabilités prédites, nous pouvons déduire le label d'un audio avec la méthode **.argmax** sur l'objet qui contient les probabilités prédites 

In [286]:
def make_predictions(model: torch.nn.Module, data:torch.utils.data.DataLoader, device: torch.device = device):
    pred_probs = []
    model.eval()
    with torch.inference_mode():
        for sample in data:
            # Preparation du sample
            sample = sample.to(device) # Envoyons sample sur GPU

            # Forward pass (le model retourne un résultat du modèle logit brut)
            pred_logit = model(sample)

            # Obtention des probabilités prédites (logit -> probabilité prédite)
            pred_prob = torch.softmax(pred_logit.squeeze(), dim=0)

            # Obtenez pred_prob du GPU pour d'autres calculs
            pred_probs.append(pred_prob.cpu())
            
    # Empiler les pred_probs pour transformer la liste en tenseur
    return torch.stack(pred_probs)

In [287]:
preds_probs = make_predictions(myModel,preds,device)
preds_probs  # Probabilités prédites par le modèle

tensor([[1.3846e-01, 8.6148e-01, 8.6221e-06,  ..., 9.1803e-06, 9.9807e-06,
         9.1424e-06],
        [9.4104e-01, 5.8840e-02, 1.6257e-05,  ..., 1.4621e-05, 1.3409e-05,
         1.8374e-05],
        [7.9159e-01, 2.0733e-01, 1.2109e-04,  ..., 9.6106e-05, 1.2486e-04,
         1.5941e-04],
        ...,
        [6.0080e-01, 3.9891e-01, 3.8994e-05,  ..., 3.9644e-05, 3.7896e-05,
         2.7837e-05],
        [6.4643e-01, 3.5302e-01, 6.1608e-05,  ..., 6.3331e-05, 2.9521e-05,
         8.4531e-05],
        [8.3328e-01, 1.6651e-01, 2.2734e-05,  ..., 2.5535e-05, 2.5625e-05,
         2.6073e-05]])

> Nous affichons les 20 premières prédictions du fichier test. 

In [290]:
# Transformez les probabilités de prédiction en label de prédiction avec argmax()
pred_classes = preds_probs.argmax(dim=1)
pred_classes[:20]

tensor([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0])

## CODE DE SAUVEGARDE DE NOS MODÈLES DE DL

In [194]:
# 1. Creation des liens de répertoire de nos modèles
MODEL_PATH = Path("/Users/nacersere/Downloads/models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

# 2. Creation d'un répertoire de sauvegarde du modèle
MODEL_NAME = "04_pytorch_workflow_model_1_cross_.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# 3. Sauvegarde du modèle
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=myModel.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH)

Saving model to: /Users/nacersere/Downloads/models/04_pytorch_workflow_model_1_cross_validation.pth


> Nous avons sauvegardé trois modèles :

- Le premier modèle est celui du gradient stochastique avec un learning_rate de 0.1 qui performe a ***87.95%***

- Le second modèle est celui du modèle Adam avec un learning rate de 0.01 qui performe a ***86%***

- Le troisème modèle est celui de la cross_validation que nous avons utilisé pour les prédictions et qui performe a ***89%***

> Liens vers les modèles : 

- SGD, learning_rate = 0.1 & performance ***87.95%*** = https://drive.google.com/file/d/1pAuGEJV15JU2dloxjyMP1v5ckBY00Sx9/view?usp=share_link

- Adam, learning_rate = 0.01 & performance ***86%*** = https://drive.google.com/file/d/1RZ81p6etXCHr5qDTknGUu2wkAdFW8qvg/view?usp=share_link

- Cross_Validation, performance ***89%*** = https://drive.google.com/file/d/1SrySL079U86sXL7yvspoH-wtI4NDgTw3/view?usp=share_link